In [62]:
from common.Cstruct import PyCStruct, FileClass
from io import BytesIO
from collections import OrderedDict, defaultdict
import binascii
import json
import struct
from enum import Enum
import os
import re
import zlib
import csv

In [2]:
def readAt(data, offset, class_def, **kw):
    tell = data.tell()
    data.seek(offset)
    ret = class_def(data, **kw)
    data.seek(tell)
    return ret

In [3]:
def find_file(file,folder):
    for root, dirs, files in os.walk(folder):
        if file in files:
            print(root+'/'+file)
            return root+'/'+file
    print('File not exits!')
    return ''

In [4]:
def varify_file(ext,folder,func,vfunc,v=None):
    ret = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if ext == file[-len(ext):]:
                path = root+'/'+file
                vret = vfunc(path, func, v)
                if vret:
                    ret += [(path,vret)]
                    if v:
                        v.write('{} {}\n'.format(path,vret))
    return ret


# Varify Function

In [5]:
def comp(path,func,v=None):
    try:
        efx = func(open(path, 'rb'))
        cmp = open(path, 'rb').read()
        if efx.serialize() != cmp:
            print(path)
            fprint(False, v)
        return True
    except:
        print(path)
        print('FAIL')
    return None

In [79]:
# all unique EFX types

v = open('v.txt','w')
for t in sorted(alltype):
    st = 'if (t == {1})\n\treturn "{0}";\n'.format(t.decode('utf-8'), int("0b"+"1"*32, 2) - zlib.crc32(t))
    v.write(st)

v.close()

# Helper

In [7]:
# TODO

class XYZ(list):
    
    def __init__(self, data, **kw):
        super().__init__(data)
        
        self.skip = 2 if kw.get('skip', False) else 1
        self.is4 = kw.get('in4', False)
        
        self.x = self[0*self.skip]
        self.y = self[1*self.skip]
        self.z = self[2*self.skip]
        self.a = self[3*self.skip] if self.is4 else None
        
        self.r = self.x
        
    def marshall(self):
        self[0*self.skip] = self.x
        self[1*self.skip] = self.y
        self[2*self.skip] = self.z
        if self.is4:
            self[3*self.skip] = self.a
        return [i for i in self]


In [8]:
def _PEEK(data, x=4):
    pos = data.tell()
    d = data.read(x)
    data.seek(pos)
    return d

In [9]:
def fprint(obj, v=None):
    print(obj) if not v else v.write(str(obj)+'\n')
    return True if v else False
    

In [10]:
def hexfy(s):
    h = [s[j*2:j*2+2] for j in range(len(s)//2)]
    c = len(h)//4
    return [' '.join(h[j*4:j*4+4]).upper() for j in range(c)] + ([' '.join(h[-(len(h)%4):]).upper()] if len(h)%4 else [])
    

In [11]:
def align(num, amount):
    return (num + (amount - 1)) & ~(amount - 1)

def pad(array, amount=16):
    padAmount = align(len(array), amount) - len(array)
    return array + b'\0' * padAmount


In [12]:
def flatten(i):
    return [k for j in i for k in flatten(j)] if type(i) == list else [i]


In [13]:
def cv(t, d):
    return struct.unpack(t,bytes.fromhex(d))[0]


In [14]:
def phrase_EOS(path, path_len):
    e_path = path.encode('utf-8')
    if path_len and not e_path:
        e_path += b'\x00'
    elif e_path and e_path[-1] != 0:
        e_path += b'\x00'
    return (e_path, len(e_path))


In [15]:
class _PATH(PyCStruct):
    fields = OrderedDict([('path_len','int')])
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path
    

In [16]:
def efx_class():
    return OrderedDict([(struct.pack('>I',k), v) for k,v in OrderedDict([
          
        (0x4A1AF914, Unkn_00            ), 
        (0x9053F04A, Unkn_02            ), 
        (0x7204060C, Unkn_05            ), 
        (0xA65E3028, Unkn_08            ), 
        (0xECC78719, Unkn_09            ),
        
        (0x9FF9D85A, Unkn_10            ), 
        (0x9695BD50, Unkn_13            ), 
        (0xB9244A04, Unkn_14            ), 
        (0x9098CF22, Unkn_15            ), 
        (0x95435914, Unkn_16            ), 
        (0x3BE6DD5C, Unkn_17            ), 
        (0xB0250E39, Unkn_18            ), 
        (0xF0F55C60, Unkn_19            ), 
        (0x5D6F6B10, Unkn_20            ), 

        (0x8F76DE00, Unkn_21            ), 
        (0x1232060F, Unkn_22            ), 
        (0x485A9229, Unkn_23            ), 
        (0x0571BB10, Unkn_24            ), 
        (0xF351EF49, Unkn_25            ), 
        (0x77995409, Unkn_26            ), 
        (0x566BA50C, Unkn_27            ), 
        (0x5B923032, Unkn_28            ), 
        (0x03796630, Unkn_29            ), 
        (0xB92C8A4B, Unkn_30            ), 
        
        (0xF4CD137E, Unkn_31            ), 
        (0xD8A29348, Unkn_32            ), 
        (0x3E538B5B, Unkn_33            ), 
        (0xE16C3D42, Unkn_34            ), 
        (0xD6A14D3F, Unkn_35            ), 
        (0x9DEF3772, Unkn_36            ), 
        (0x5FB0E810, Unkn_37            ), 
        (0x029C6632, Unkn_38            ), 
        (0xEFE81F4D, Unkn_39            ), 
        
        (0xF8A71372, Unkn_40            ), 
        (0xD442D203, Unkn_41            ), 
        (0xCCFBD20C, Unkn_42            ), 
        (0x4693BE0A, Unkn_43            ), 
        (0x88601426, Unkn_44            ), 

        (0xADF69C00, Basic_Transform    ),
        (0xCA47F215, Limit_Transform    ),
        
        (0xACCF8B72, Launcher_Prop      ),
        (0x84DABA4E, Life_Cycle         ),
        (0xD1A9D43B, Symmetric_Copy     ),
        (0xD472420D, Momentum_Control   ),
        (0xCC023733, Visible_Dist       ),
        (0x8E1B0913, Dds_Trace_Prop     ),
        (0xDEC8C343, Dds_Useage         ),
        (0x8844A21C, Loop_Play          ),
        (0x493A4465, Uvs_Useage         ),
        (0x2F24A603, Transparent_Control),
        (0x5AF8E975, Layer_Visibility   ),
        (0xEA9A641B, Color_Prop_Control ),
        (0x8DA67D10, Mod3_Useage        ),
        (0x0546BF69, Self_Spin_Control  ),
        (0xDB859B23, Auras              ),
        (0x07CCEF42, Follow             ),
            
        (0xB32C4746, EFX_Behavior       ),
        (0x6BB9E262, Texture            ),
        (0xCDDB4102, Flowmap            ),
        (0x8395F662, Cubemap            ),
        (0xB204E037, CM_Map             ),
            
        (0xD916257E, Unkn_01            ),
        (0xF223B52B, Unkn_03            ),
        (0x62962C1F, Unkn_04            ),
        (0x86CE6778, Unkn_06            ),
        (0xAC5B1549, Unkn_07            ),
            
        (0xE8950F34, Unkn_11            ),
        (0x2456671D, Unkn_12            ),
    ]).items()])

# EFX Header

In [17]:
class EFX(PyCStruct):
    fields = OrderedDict([
        ('EFX', 'char[4]'),
        ('VERSION', 'long'),
        ('CONST0', 'long[5]'),
        ('EFXR', 'char[4]'),
        ('CONST1', 'int'),
        ('unkn1', 'long'),
        ('EFX_Count', 'int'),
        ('EFX_type_len', 'int'),
        ('IE_Count', 'int'),
        ('UNKN_Count', 'int'),
        ('NULL_Count', 'int'),
        ('NULL_Lenth', 'int'),
        ('End_Count', 'int'),
        ('Unkn_Offset', 'int')
    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        
        self.efx_type = []
        for _ in range(self.EFX_Count + self.IE_Count + self.UNKN_Count):
            self.efx_type += [data.read(_PEEK(data,256).find(0)+1)]
        data.read(self.NULL_Count)

        self.ie = []
        for _ in range(self.IE_Count):
            self.ie += [IEPrefix(data)]
            
        self.un = []
        for _ in range(self.UNKN_Count):
            self.un += [UN_Prefix(data)]
        
        self.main = []
        for _ in range(self.EFX_Count):
            if struct.unpack('i',_PEEK(data))[0] == 1228515738: # 9A A9 39 49
                self.main += [Single_Prefix(data)]
            else:
                self.main += [Main_Prefix(data)]
                
        self.null = []
        for _ in range(self.NULL_Count):
            self.null += [NULL_Entry(data)]
            
        self.end = struct.unpack('i'*self.End_Count,data.read(4*self.End_Count))
        
        # remain (if there is)
        self.peek = _PEEK(data,256)

            
    def serialize(self):
        ret = super().serialize()
        ret += b''.join(self.efx_type)
        ret += b'\x00' * self.NULL_Count
        ret += b''.join([ie.serialize() for ie in self.ie])
        ret += b''.join([un.serialize() for un in self.un])
        ret += b''.join([main.serialize() for main in self.main])
        ret += b''.join([null.serialize() for null in self.null])
        ret += b''.join([struct.pack('i', end) for end in self.end])
        
        return ret
    

# Overall Prefix

## External / Internal

In [18]:
class IEPrefix(PyCStruct):
    
    class External(PyCStruct):
        fields = OrderedDict([
            ('External_Header', 'long'),
            ('unkn0', 'int'),
            ('path_len', 'int'),
            ('External', 'long'),
            ('unkn1', 'int[7]'),
            ('XYZ', 'float[3]'),
            ('unkn2', 'int[3]'),
        ])
            
        def __init__(self, data, **kw):
            super().__init__(data, **kw)
            self.path = data.read(self.path_len).decode('utf-8')
        
        def serialize(self):
            (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
            return super().serialize() + e_path
            
    class Internal(PyCStruct):
        fields = OrderedDict([
            ('Internal', 'long'),
            ('unkn1', 'int[7]'),
            ('XYZ', 'float[3]'),
            ('unkn2', 'int[3]'),
            ('Attri_Count', 'int')
        ])
        
        def __init__(self, data, **kw):
            super().__init__(data, **kw)
            self.Attri = []
            for i in range(self.Attri_Count):
                self.Attri += [struct.unpack('i', data.read(4))[0]]
                
        def serialize(self):
            return super().serialize() + b''.join([struct.pack('i',at) for at in self.Attri])

    fields = OrderedDict([
        ('Prefix_Header', 'long'),
        ('Prefix_Len', 'int')
    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        buffer = data.read(4)
        data.seek(data.tell()-4)
        self.external = None
        self.internal = None

        if not self.Prefix_Len:
            pass
        elif buffer == b'\x2F\xED\x2B\x75':
            if self.Prefix_Len >= 1:
                self.external = IEPrefix.External(data)
            if self.Prefix_Len == 2:
                self.internal = IEPrefix.Internal(data) 
        elif buffer == b'\x25\x31\xAF\x44':
            if self.Prefix_Len != 1:
                print('len ERROR')
            self.external = None
            self.internal = IEPrefix.Internal(data)
        else:
            print(self.Prefix_Len)
            print('type ERROR')
            self.Prefix_Len = 233
        
    def serialize(self):
        return super().serialize() + \
            (self.external.serialize() if self.external else b'') + \
            (self.internal.serialize() if self.internal else b'')


## Unknown Prefix

In [19]:
class UN_P(PyCStruct):
    class UN_P_0  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[57]'),
        ])

    class UN_P_1  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[27]'),
        ])

    class UN_P_2  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[19]'),
        ])

    class UN_P_3  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[28]'),
        ])

    class UN_P_4  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[18]'),
        ])

    class UN_P_5  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[44]'),
            ('unkn1', 'byte'),
        ])

    class UN_P_6  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[33]'),
            ('unkn1', 'byte'),
        ])

    class UN_P_7  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[22]'),
        ])

    class UN_P_8  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[11]'),
            ('unkn1', 'byte'),
        ])

    class UN_P_9  (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[19]'),
        ])

    class UN_P_10 (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[12]'),
        ])

    class UN_P_11 (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[90]'),
            ('unkn1', 'byte'),
        ])

    class UN_P_12 (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[21]'),
        ])
        
    # Class 13 = EFX Behaiv

    class UN_P_14 (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[40]'),
            ('unkn1', 'byte'),
        ])

    class UN_P_15 (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[18]'),
        ])

    class UN_P_16 (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[20]'),
        ])
        
    class UN_P_17 (PyCStruct):
        fields = OrderedDict([
            ('unkn0', 'long[39]'),
            ('unkn1', 'byte'),
        ])

    fields = OrderedDict([
        ('UN_P_Attribute', 'long'),
        ('unkn','long'),
        ('Attribute_Count', 'int'),
    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
#         print(self.UN_P_Attribute, self.unkn, self.Attribute_Count)
        if self.UN_P_Attribute == 500644368:    # 10 3A D7 1D
            ATTR = UN_P.UN_P_0
        elif self.UN_P_Attribute == 351887441 :  # 51 60 F9 14
            ATTR = UN_P.UN_P_1
        elif self.UN_P_Attribute == 786529163 :  # 8B 7B E1 2E
            ATTR = UN_P.UN_P_2
        elif self.UN_P_Attribute == 2069124466:  # 72 55 54 7B
            ATTR = UN_P.UN_P_3
        elif self.UN_P_Attribute == 28559457  :  # 61 C8 B3 01
            ATTR = UN_P.UN_P_4
        elif self.UN_P_Attribute == 1850314036:  # 34 8D 49 6E
            ATTR = UN_P.UN_P_5
        elif self.UN_P_Attribute == 693979274 :  # 8A 48 5D 29
            ATTR = UN_P.UN_P_6
        elif self.UN_P_Attribute == 1880343637:  # 55 C4 13 70
            ATTR = UN_P.UN_P_7
        elif self.UN_P_Attribute == 2097096908:  # CC 28 FF 7C
            ATTR = UN_P.UN_P_8
        elif self.UN_P_Attribute == 725249589 :  # 35 6E 3A 2B
            ATTR = UN_P.UN_P_9
        elif self.UN_P_Attribute == 1338793878:  # 96 5F CC 4F
            ATTR = UN_P.UN_P_10
        elif self.UN_P_Attribute == 839790967 :  # 77 31 0E 32
            ATTR = UN_P.UN_P_11
        elif self.UN_P_Attribute == 283026906 :  # DA A5 DE 10
            ATTR = UN_P.UN_P_12
        elif self.UN_P_Attribute == 1610366518:  # 36 3E FC 5F
            ATTR = EFX_Behavior_Body
        elif self.UN_P_Attribute == 482524730 :  # 3A BE C2 1C
            ATTR = UN_P.UN_P_14
        elif self.UN_P_Attribute == 705591903 :  # 5F 7A 0E 2A
            ATTR = UN_P.UN_P_15
        elif self.UN_P_Attribute == 1879331968:  # 80 54 04 70
            ATTR = UN_P.UN_P_16
        elif self.UN_P_Attribute == 805496014 :  # CE E4 02 30
            ATTR = UN_P.UN_P_17
        else:
            print('UN_Prefix Attribute Type Error')
            print(self.UN_P_Attribute, data.tell())
            self.UN_P_Attribute = 0
            self.attributes = []
            return
        
        self.attributes = []
        for _ in range(self.Attribute_Count):
            self.attributes += [ATTR(data)]
            
    def serialize(self):
        return super().serialize() + b''.join([attr.serialize() for attr in self.attributes])
    

class UN_Prefix(PyCStruct):
    
    fields = OrderedDict([
        ('Prefix_Header', 'long'),
        ('NULL0','long'),
        ('Prefix_Count', 'int'),
        ('NULL1','long'),

    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        self.unknown_prefix = []
        for _ in range(self.Prefix_Count):
#             print(self.Prefix_Header, self.NULL0, self.Prefix_Count)
            self.unknown_prefix += [UN_P(data)]
    
    def serialize(self):
        return super().serialize() + b''.join([unp.serialize() for unp in self.unknown_prefix])
    

# EFX Main Body

## Single Prefix

In [20]:
class Single_Prefix(PyCStruct):
    
    class S_1(PyCStruct):
        fields = OrderedDict([
            ('Single_Prefix_Attribute', 'long'),
            ('unkn0', 'long[10]'),
        ])
    
    class S_2(PyCStruct):
        
        class S_2_E(PyCStruct):
            fields = OrderedDict([
                ('NULL', 'long'),
                ('unkn0', 'float[6]'),
                ('unkn1', 'float[9]'),
            ])
        
        fields = OrderedDict([
            ('Single_Prefix_Attribute', 'long'),
            ('Path_Count', 'int'),
        ])
        
        def __init__(self, data=None, **kw):
            super().__init__(data, **kw) # CCC length
            self.paths = []
            for _ in range(6):
                self.paths += [_PATH(data)]
            self.s_2_e = Single_Prefix.S_2.S_2_E(data)
                        
        def serialize(self):
            return super().serialize() +  b''.join([path.serialize() for path in self.paths]) + self.s_2_e.serialize()
        
    class S_3(PyCStruct):
        
        class S_3_Block(PyCStruct):
            
            class S_3_B(PyCStruct):
                fields = OrderedDict([])
                def __init__(self, data=None, **kw):
                    peek = kw['peek']
                    count = kw['count']
                    if peek > 0 and peek < 6:
                        self.fields = OrderedDict([('Entry_Type', 'int'),('Entry','float[{}]'.format(count*2))])
                    elif peek == 0 or peek == 6:
                        self.fields = OrderedDict([('Entry_Type', 'int'),('Entry','float[{}]'.format(count*3))])
                    elif peek == 7:
                        type7_count = struct.unpack('i',_PEEK(data,8)[-4:])[0]
                        self.fields = OrderedDict([('Entry_Type', 'int'),
                                                   ('Type7_Count','int'), 
                                                   ('Entry','long[{}]'.format(count*2*type7_count))])
                    else:
                        print('S_3_B Type Error')
                    super().__init__(data, **kw)
            
            fields = OrderedDict([
                ('Entry_Count', 'int'),
            ])
            
            def __init__(self, data=None, **kw):
                super().__init__(data, **kw)
                self.entry = []
                if self.Entry_Count:
                    while struct.unpack('i',_PEEK(data))[0] != -1:
                        self.entry += [Single_Prefix.S_3.S_3_Block.S_3_B(data,count=self.Entry_Count,peek=struct.unpack('i',_PEEK(data))[0])]
                    data.read(4)
                
            def serialize(self):
                if self.Entry_Count:
                    return super().serialize() +  b''.join([entry.serialize() for entry in self.entry]) + b'\xFF'*4
                else:
                    return super().serialize()
        
        fields = OrderedDict([
            ('Single_Prefix_Attribute', 'long'),
            ('unkn0', 'int'),
            ('Block_Count', 'int')
        ])
        def __init__(self, data=None, **kw):
            super().__init__(data, **kw)
            self.blocks = []
            for _ in range(self.Block_Count):
                self.blocks += [Single_Prefix.S_3.S_3_Block(data)]
        def serialize(self):
            return super().serialize() +  b''.join([blocks.serialize() for blocks in self.blocks])
    
    fields = OrderedDict([
        ('Prefix_Header', 'long'),
        ('CONST1', 'int'), # 01 00 00 00
        ('Attribute_Count', 'int'),
        ('CONST2', 'int'), # 00 00 00 00
    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        self.attributes = []
        for _ in range(self.Attribute_Count):
            peek = struct.unpack('i',_PEEK(data))[0]
            if peek == 1413509420: # 2C 71 40 54
                self.attributes += [Single_Prefix.S_1(data)]
            elif peek == 2083659062: # 36 1D 32 7C
                self.attributes += [Single_Prefix.S_2(data)]
            elif peek == 2050487542: # F6 F4 37 7A
                self.attributes += [Single_Prefix.S_3(data)]
            else:
                print('Single_Prefix Attribute Type Error')
                print(hexfy(_PEEK(data,256).hex()))
        
    def serialize(self):
        return super().serialize() + b''.join([attr.serialize() for attr in self.attributes])
    

## Main Prefix

In [21]:
class Main_Prefix(PyCStruct):
    fields = OrderedDict([
        ('Prefix_Header', 'long'),
        ('unkn1', 'int'), # 01 00 00 00
        ('Attribute_Count', 'int'),
        ('unkn2', 'int'), # 00 00 00 00
        ('TIML_Length', 'int'),
    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        if self.TIML_Length:
            self.timl = TIML(data,TIML_Length = self.TIML_Length)
        self.attributes = []
        for _ in range(self.Attribute_Count):
            peek = _PEEK(data)
            if peek not in efx_classes:
                print('Main Pefix Type Error')
                print(data.tell(), hexfy(peek.hex()))
                break
            self.attributes += [efx_classes[peek](data)]
            
    def serialize(self):
        return super().serialize() + (self.timl.serialize() if self.TIML_Length else b'') + \
                    b''.join([attr.serialize() for attr in self.attributes])

## TIML

In [22]:
class TIML(PyCStruct):
    
    class Time(PyCStruct):
            
        class Set0(PyCStruct):
            fields = OrderedDict([
                ('Offset','int64'),
                ('Set1_Count','int64'),
                ('unkn1','int'),
                ('unkn2','int'),
                ('unkn3','float'),
                ('unkn4','int'),
                ('unkn5','int'),
                ('unkn6','long'),
            ])
        class Set1(PyCStruct):
            fields = OrderedDict([
                ('Offset','int64'),
                ('Set2_Count','int64'),
                ('unkn0','int64'),
            ])
            
        class Set2s(PyCStruct):
            
            class Set2(PyCStruct):

                class Foot(PyCStruct):
                    fields = OrderedDict([
                        ('key_trans_rot','long'),
                        ('end_thresh','long'),
                        ('start_thresh','float'),
                        ('key_reg','float'),
                        ('displacement','short'),
                        ('rotate','short'),
                    ])

                fields = OrderedDict([
                    ('Offset','int64'),
                    ('Foot_Count','int64'),
                    ('unkn0','long'),
                    ('unkn1','int'),
                ])

                def __init__(self, data=None, **kw):
                    super().__init__(data, **kw)
    #                 print('set2', data.tell())
                    offset = data.tell()
                    self.foot = []
                    data.seek(kw['base_offset']+self.Offset)
                    for _ in range(self.Foot_Count):
                        self.foot += [TIML.Time.Set2s.Set2.Foot(data)]
                    data.seek(offset)

                def clean_up(self, local_offset, end=False):
                    self.Offset = local_offset
                    self.Foot_Count = len(self.foot)
                    if end:
                        return (self.serialize(), b''.join([foot.serialize() for foot in self.foot]))
                    else:
                        return (self.serialize(), pad(b''.join([foot.serialize() for foot in self.foot])))
            
            def __init__(self, data=None, **kw):
                self.entry = []
                for _ in range(kw['count']):
                    self.entry += [TIML.Time.Set2s.Set2(data,base_offset = kw['base_offset'])]
                    
            def clean_up(self, local_offset, end=False):
                count = len(self.entry)
                all_foot = b''
                all_set2 = b''
                for i in range(count):
                    (set2, foot) = self.entry[i].clean_up(local_offset, end and (i == count-1))
                    all_set2 += set2
                    all_foot += foot

                    local_offset += len(foot)
                return (pad(all_set2), all_foot)

            def serialize(self):
                return pad(b''.join([set2.serialize() for set2 in self.entry]))

        fields = OrderedDict([])
        
        def __init__(self, data=None, **kw):
            self.base_offset = kw['base_offset']
            self.time_offset = kw['time_offset']
            if not self.time_offset:
                return

            data.seek(self.base_offset + self.time_offset)
            
            self.set0 = TIML.Time.Set0(data)
            # print(self.set0.unkn0)

            offset = self.base_offset+self.set0.Offset
            self.set1 = []

            for _ in range(self.set0.Set1_Count):
                data.seek(offset)
                self.set1 += [TIML.Time.Set1(data)]
                offset += len(self.set1[0].serialize())

            self.set2 = []
            for i in range(self.set0.Set1_Count):
                data.seek(self.base_offset + self.set1[i].Offset)
                self.set2 += [TIML.Time.Set2s(data,base_offset = self.base_offset, count = self.set1[i].Set2_Count)]

        def clean_up(self, local_offset, end=False):
            if not self.time_offset:
                return b''
#             print('set0 off', local_offset)
            self.set0.Offset = local_offset + len(pad(self.set0.serialize()))
            self.set0.Set1_Count = len(self.set1)
#             print('set1 off', self.set0.Offset)
            local_offset = self.set0.Offset + len(pad(b''.join([set1.serialize() for set1 in self.set1])))

            for i in range(self.set0.Set1_Count):
                self.set1[i].Offset = local_offset
                self.set1[i].Set2_Count = len(self.set2[i].entry)

                local_offset += len(self.set2[i].serialize())

            all_set2 = b''
            all_foot = b''
            for i in range(self.set0.Set1_Count):
#                 print('foot off', local_offset)
                (set2, foot) = self.set2[i].clean_up(local_offset, end and (i ==self.set0.Set1_Count-1))
#                 print('foot', hexfy(foot.hex()))
                local_offset += len(foot)
                all_set2 += set2
                all_foot += foot
                
#             print('all set 2',hexfy((pad(all_set2) + all_foot).hex()))
            return pad(self.set0.serialize()) + pad(b''.join([set1.serialize() for set1 in self.set1])) + pad(all_set2) + all_foot
    
    fields = OrderedDict([
        ('type', 'long'),
        ('CONST1', 'long[5]'),
        ('Time_Count', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        self.base_offset = data.tell()
#         print('TIML', self.base_offset)
        super().__init__(data, **kw)
        if self.Time_Count:
            data.read(4)
        self.time_offset = [struct.unpack('q',data.read(8))[0] for _ in range(self.Time_Count)]
        
        self.time = []
        for offset in self.time_offset:
            self.time += [TIML.Time(data,base_offset = self.base_offset, time_offset = offset)]
#         print(self.time[-1].set2[1].serialize())
        try:
            data.seek(self.base_offset+kw['TIML_Length'])
        except:
            pass
            
    def serialize(self):
        self.Time_Count = len(self.time)
        
        local_offset = len(pad(super().serialize()) + pad(self.Time_Count * 8 *b'\x00'))
#         print('local 1', local_offset)
        
        all_time = b''
        for i in range(self.Time_Count):
#             print('time off', local_offset, i == self.Time_Count-1)
            time = self.time[i].clean_up(local_offset, i == self.Time_Count-1)
            
            if not len(time):
                self.time_offset[i] = 0
            else:
                self.time_offset[i] = local_offset
                local_offset += len(time)
            all_time += time        
#         print(self.time_offset)
        return (pad(super().serialize()) if self.Time_Count else super().serialize()) + \
                    pad(b''.join([struct.pack('q', q) for q in self.time_offset])) + all_time
    

## Main Body Type

In [35]:
# Reload class
efx_classes = efx_class()


In [24]:
class Unkn_00(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[9]'),
    ])
    
class Unkn_02(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'byte[4]'),
        ('unkn2' ,'float'),
        ('unkn3' ,'int'),
        ('unkn4' ,'long'),
        ('unkn5' ,'float[9]'),
    ])

class Unkn_05(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'byte'),
    ])

class Unkn_08(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[10]'),
    ])

class Unkn_09(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int64[2]'),
        ('unkn1' ,'float[2]'),
    ])

class Unkn_10(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'long[2]'),
        ('unkn2' ,'float[2]'),
        ('unkn3' ,'int[4]'),
        ('unkn4' ,'float[16]'),
        ('path_len' ,'int'),
        ('unkn5' ,'int[2]'),
    ])        
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

class Unkn_13(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'float[11]'),
    ])

class Unkn_14(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'long'),
        ('unkn1' ,'float[3]'),
    ])

class Unkn_15(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int'),
        ('unkn1' ,'float[4]'),
        ('unkn2' ,'int[4]'),
    ])

class Unkn_16(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'float[9]'),
        ('unkn2' ,'int'),
        ('unkn3' ,'float[3]'),
        ('unkn4' ,'int[3]'),
    ])

class Unkn_17(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn00' ,'int[2]'),
        
        ('unkn01' ,'long'),
        ('color1' ,'byte[4]'),
        ('unkn02' ,'long'),
        ('color2' ,'byte[4]'),
        ('unkn03' ,'long'),
        ('color3' ,'byte[4]'),
        ('unkn04' ,'long'),
        
        ('unkn05' ,'float[49]'),
        ('unkn06' ,'int[2]'),
        ('unkn07' ,'float[28]'),
        ('unkn08' ,'int[2]'),
        ('unkn09' ,'float[20]'),
        ('unkn10' ,'int[4]'),
        ('unkn11' ,'float[2]'),
        ('unkn12' ,'int[2]'),
        ('unkn13' ,'float[6]'),
        ('unkn14' ,'int[3]'),
        ('unkn15' ,'float[9]'),
        ('unkn16' ,'short'),
        ('path_len' ,'int'),
    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

class Unkn_18(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[3]'),
    ])

class Unkn_19(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int'),
    ])

class Unkn_20(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'float[5]'),
        ('unkn2' ,'long[3]'),
        ('unkn3' ,'float[9]'),
        ('unkn4' ,'short'),
    ])

class Unkn_21(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'long[18]'),
    ])

class Unkn_22(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[3]'),
        ('unkn1' ,'float[11]'),
        ('unkn2' ,'int[2]'),
        ('unkn3' ,'long[4]'),
        ('unkn4' ,'float[4]'),
        ('unkn5' ,'int[2]'),
        ('unkn6' ,'long[4]'),
        ('unkn7' ,'float'),
        ('path_len' ,'int'),
    ])
    
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

class Unkn_23(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'float[4]'),
        ('unkn2' ,'int[2]'),
        ('unkn3' ,'float'),
    ])

class Unkn_24(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[8]'),
        ('unkn1' ,'float[3]'),
        ('unkn2' ,'int[2]'),
        ('unkn3' ,'float[9]'),
        ('unkn4' ,'int[2]'),
        ('unkn5' ,'float'),
        ('unkn6' ,'int[3]'),
    ])

class Unkn_25(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'long[17]'),
        ('unkn1' ,'short'),
    ])


    

In [25]:
class Unkn_27(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('length', 'int'),
        ('unkn1', 'long' ),
        ('unkn2', 'float[3]'),
        ('unkn3', 'int[2]'),
    ])
    
    def __init__(self, data=None, **kw):
        if struct.unpack('i'*3,_PEEK(data,12))[2] % 4 != 0:
            print('UNKN 27 Size Error')
        self.fields['unkn1'] = 'long[{}]'.format(struct.unpack('i'*3,_PEEK(data,12))[2]//4-5)
        super().__init__(data, **kw)
        
class Unkn_28(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('length', 'int'),
        ('unkn1', 'long' ),
        ('unkn2', 'float[3]'),
        ('unkn3', 'int[2]'),
    ])
    
    def __init__(self, data=None, **kw):
        if struct.unpack('i'*3,_PEEK(data,12))[2] % 4 != 0:
            print('UNKN 28 Size Error')
        self.fields['unkn1'] = 'long[{}]'.format(struct.unpack('i'*3,_PEEK(data,12))[2]//4-5)
        super().__init__(data, **kw)
        

In [26]:
class Unkn_29(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[3]'),
    ])

class Unkn_30(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'long'),
        ('unkn2' ,'int[2]'),
        ('unkn3' ,'long[3]'),
        ('unkn4' ,'float[4]'),
        ('unkn5' ,'int'),
        ('unkn6' ,'float[8]'),
    ])

In [27]:
class Unkn_31(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'int'),
    ])

class Unkn_32(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'float'),
        ('unkn3', 'int'),
        ('unkn4', 'float[6]'),
        ('unkn5', 'int[8]'),
        ('unkn6', 'byte'),
    ])

class Unkn_33(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'float[10]'),
    ])

# Mod 3
class Unkn_34(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long[3]'),
        ('unkn2', 'byte[8]'),
        ('unkn3', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        n = _PEEK(data,256).find(0)+1
        self.mod3_path = data.read(n).decode('utf-8')
                    
    def serialize(self):
        return super().serialize() + phrase_EOS(self.mod3_path,1)[0]
    
# NM Useage
class Unkn_35(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn00', 'int[2]'),
        ('unkn01', 'long'),
        ('color1', 'byte[4]'),
        ('unkn02', 'long'),
        ('color2', 'byte[4]'),
        ('unkn03', 'float[20]'),
        ('unkn04', 'int[3]'),
        ('unkn05', 'float[3]'),
        ('color3', 'byte[4]'),
        ('unkn06', 'float[25]'),
        ('unkn07', 'int[6]'),
        ('unkn08', 'float[6]'),
        ('unkn09', 'int[6]'),
        ('unkn10', 'float[3]'),
        ('unkn11', 'int'),
        ('unkn12', 'float[4]'),
        ('unkn13', 'int'),
        ('path_len', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

# 2 cases
class Unkn_36(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'float[1]'),
        ('unkn3', 'int'),
        ('unkn4', 'short'),
    ])

class Unkn_37(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'int[7]'),
    ])

# 1 case CC_Effect cm_flash_000
class Unkn_38(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'float[3]'),
        ('path_len', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

class Unkn_39(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[4]'),
        ('unkn1', 'float[168]'),
    ])

In [28]:

class Unkn_40(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'float'),
        ('unkn3', 'int'),
    ])

# 1 case
class Unkn_41(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'float[3]'),
    ])

class Unkn_42(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'int'),
        ('color', 'byte[4]'),
        ('unkn3', 'long[2]'),
        ('unkn4', 'float[13]'),
    ])

# 2 cases
class Unkn_43(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'long'),
        ('unkn2', 'int[2]'),
        ('color', 'byte[4]'),
        ('unkn3', 'int'),
        ('unkn4', 'long'),
        ('unkn5', 'float[4]'),
        ('unkn6', 'long'),
        ('unkn7', 'float[8]'),
    ])

# 1 case
class Unkn_44(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'float'),
    ])



In [29]:
class Basic_Transform(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('b_local_coord', 'float[6]'), # XYZ # EXCEPTION see template
        ('b_rotate', 'float[6]'),      # XYZ
        ('b_resize', 'float[6]'),      # XYZ
        ('unkn1', 'int'),
        ('b_unkn2', 'float[36]'),      # XYZ
        ('unkn3', 'int'),
    ])
    

In [30]:
class Limit_Transform(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('b_axis_lim', 'int[3]'),  # XYZ
        ('b_angle_lim', 'int[3]'), # XYZ
        ('unkn0', 'int[3]'),       # XYZ? Need Check
        ('efx_pos_lim_1', 'int'),
        ('unkn1', 'int'),
        ('efx_pos_lim_2', 'int'),
        ('unkn2', 'int'),
        ('efx_pos_lim_3', 'int'),
        ('bone_lim', 'int')
    ])
    

In [31]:
class Launcher_Prop(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[10]'),
        ('occur', 'int64'),
        ('unkn1', 'uint64'),
        ('unkn2', 'uint64'),
        ('unkn3', 'uint64')
    ])

In [32]:
class Life_Cycle(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('unkn1', 'long[2]'),
        ('duration', 'int[2]'),
        ('unkn2', 'int[2]'),
        ('unkn_count', 'int'),
        ('unkn3', 'int[3]'),
        ('NULL', 'int')
    ])

class Symmetric_Copy(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('unkn1', 'float[6]'),
        ('unkn2', 'int[2]'),
        ('unkn3', 'float[12]'),
        ('unkn4', 'int')
    ])

class Momentum_Control(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[3]'),
        ('unkn1', 'float[2]'),
        ('NULL0', 'long[4]'),
        ('unkn2', 'float[13]'),
        ('NULL1', 'long[5]')
    ])

class Visible_Dist(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('param', 'float[4]')
    ])

class Dds_Trace_Prop(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn00', 'int'),
        ('unkn01', 'int'),
        ('unkn02', 'long'),
        ('unkn03', 'int'),
        ('unkn04', 'float'),
        ('unkn05', 'int[2]'),
        ('unkn06', 'long'),
        ('unkn07', 'int[2]'),
        ('unkn08', 'float[5]'),
        ('unkn09', 'long'),
        ('unkn10', 'int'),
        ('unkn11', 'float'),
        ('unkn12', 'int[3]'),
        ('unkn13', 'float'),
        ('NULL1', 'long'),
        ('b_color1', 'ubyte[4]'), # RGBA
        ('NULL2', 'long'),
        ('b_color2', 'ubyte[4]'), # RGBA
        ('unkn14', 'long'),
        ('unkn15', 'int'),
        ('unkn16', 'float'),
        ('unkn17', 'int'),
        ('unkn18', 'float'),
        ('NULL3', 'long'),
        ('b_color3', 'ubyte[4]'), # RGBA
        ('NULL4', 'long'),
        ('b_color4', 'ubyte[4]'), # RGBA
        ('unkn19', 'int[2]'),
        ('unkn20', 'float'),
        ('unkn21', 'int'),
        ('unkn22', 'long'),
        ('unkn23', 'float'),
        ('NULL5', 'long'),
        ('unkn24', 'float'),
        ('NULL6', 'long'),
        ('unkn25', 'float'),
        ('NULL7', 'long'),
        ('unkn26', 'float'),
        ('NULL8', 'long'),
        ('NULL9', 'short'),
        ('path_len', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path
    
class Dds_Useage(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('color', 'ubyte[8]'),    # RGBA
        ('brightness', 'float'),
        ('unkn2', 'int[5]'),
        ('unkn3', 'float[2]'),
        ('scale', 'float[2]'),
        ('height', 'float'),
        ('NULL', 'float'),
        ('width', 'float'),
        ('unkn4', 'float[9]'),
        ('path_len', 'int'),
        ('unkn5', 'int'),
        ('unkn6', 'uint64'),
        ('unkn7', 'float'),
        ('unkn8', 'int'),
        ('unkn9', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

class Flowmap(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('color', 'ubyte[8]'),    # RGBA
        ('brightness', 'float'),
        ('unkn2', 'int[5]'),
        ('unkn3', 'float[2]'),
        ('scale', 'float[2]'),
        ('height', 'float'),
        ('NULL', 'float'),
        ('width', 'float'),
        ('unkn4', 'float[9]'),
        ('path_len', 'int'),
        ('unkn5', 'int[6]'),
        ('unkn6', 'float[4]'),
        ('unkn7', 'uint64'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

class Cubemap(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('color', 'ubyte[8]'),    # RGBA
        ('unkn1', 'float[10]'),
        ('unkn2', 'int[26]'),
        ('path_len', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path
    
class Loop_Play(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('speed', 'float'),
        ('NULL', 'long'),
        ('unkn2', 'float[16]') # 4*4
    ])

class Uvs_Useage(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('dds_template_id', 'int'),
        ('NULL', 'long'),
        ('unkn2', 'int'),
        ('unkn3', 'int'),
        ('unkn4', 'float[4]'),
        ('unkn5', 'int'),
        ('path_len', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.path = data.read(self.path_len).decode('utf-8')
    
    def serialize(self):
        (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
        return super().serialize() + e_path

class Transparent_Control(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('unkn1', 'float'),
        ('transparentness', 'float'),
        ('NULL', 'long'),
        ('unkn2', 'int')
    ])

class Layer_Visibility(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('outer_layer_radius', 'float'),
        ('unkn1', 'int'),
        ('inner_layer_radius', 'float'),
        ('unkn2', 'int'),
        ('effect_contol', 'int'),
        ('unkn3', 'int[3]'),
        ('unkn4', 'float[17]'),
        ('unkn5', 'int[3]')
    ])

class Color_Prop_Control(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('unkn1', 'long'),
        ('unkn2', 'float'),
        ('unkn3', 'long'),
        ('unkn4', 'float[4]'),
        ('unkn5', 'int'),
        ('NULL0', 'long[2]'),
        ('unkn6', 'int[17]')
    ])

class Mod3_Useage(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('CDs', 'long'),
        ('unkn2', 'float[5]'),
        ('unkn5', 'float[14]'),
        ('path', 'float'),
        ('NULL0', 'int64[2]'),
        ('unkn3', 'long[2]'),
        ('unkn7', 'int[4]'),
        ('unkn4', 'int[10]'),
        ('NULL1', 'short'),
        ('BeginMod3', 'byte')
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.mod3_path = []
        for _ in range(2):
            n = _PEEK(data,256).find(0)+1
            self.mod3_path += [data.read(n).decode('utf-8')]
                    
    def serialize(self):
        return super().serialize() + b''.join([phrase_EOS(p,1)[0] for p in self.mod3_path])

class Self_Spin_Control(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('NULL', 'long[2]'),
        ('unkn1', 'float[16]')
    ])


class EFX_Behav(PyCStruct):
    
    class EFXB_Type(PyCStruct):
        fields = OrderedDict([])
        
        def __init__(self, data=None, **kw):
            self.t = kw.get('t', 0)
            if self.t == 0x03:
                self.fields = OrderedDict([('NULL', 'long')])
            elif self.t == 0x06:
                self.fields = OrderedDict([('unkn1', 'int')])
            elif self.t == 0x05:
                self.fields = OrderedDict([('unkn1', 'short')])
            elif self.t == 0x0C:
                self.fields = OrderedDict([('unkn1', 'float')])
            elif self.t == 0x0F:
                self.fields = OrderedDict([('color', 'ubyte[4]')]) # RGBA
            elif self.t == 0x14:
                self.fields = OrderedDict([('unkn1', 'float[3]')]) # XYZ
            elif self.t == 0x15:
                self.fields = OrderedDict([
                    ('unkn0', 'float'),
                    ('unkn1', 'long'),
                    ('unkn2', 'float'),
                    ('unkn3', 'long'),
                ])
            elif self.t == 0x36 or self.t == 0x37:
                self.fields = OrderedDict([('unkn1', 'int[2]')])
            elif self.t == 0x40:
                self.fields = OrderedDict([('unkn1', 'int64')])
            elif self.t == 0x80:
                self.fields = OrderedDict([
                    ('file_type', 'long'),
                    ('path_len', 'int'),
                ])
            else:
                print('EFX Behav Block Type Error')
                print(self.t)

            super().__init__(data, **kw)

            if self.t == 0x80:
                self.path = data.read(self.path_len).decode('utf-8')

        def serialize(self):
            e_path = b''
            if self.t == 0x80:
                (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
            return super().serialize() + e_path
            
    
    fields = OrderedDict([
        ('unkn', 'long'),
        ('const0', 'long'),
        # 0x00 - ?; 0x03 - NULL; 0x06 - int; 0x09 - ?; 0x0C - float; 0x0F - Color;
        # 0x80 - file; 0x36/0x37 - int*2; 0x14 - XYZ
        ('t', 'int')
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.efxb_type = EFX_Behav.EFXB_Type(data, t=self.t)
        
    def serialize(self):
        return super().serialize() + self.efxb_type.serialize()

    
class EFX_Behavior_Body(PyCStruct):
    fields = OrderedDict([
        ('unkn0', 'int'),
        ('behav_type_len', 'int'),
        ('para_count', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.b_type = data.read(self.behav_type_len).decode('utf-8')
        self.efx_behav = []
        for i in range(self.para_count):
            self.efx_behav += [EFX_Behav(data)]
        
    def serialize(self):
        (e_btype, self.behav_type_len) = phrase_EOS(self.b_type, self.behav_type_len)
        return super().serialize() + e_btype + b''.join([i.serialize() for i in self.efx_behav])

class EFX_Behavior(PyCStruct):
    fields = OrderedDict([
        ('type', 'long')
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.body = EFX_Behavior_Body(data, **kw)
        
    def serialize(self):
        return super().serialize() + self.body.serialize()
    
    
class Auras(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'float'),
        ('body_p', 'ubyte'),
        ('wp_p', 'ubyte'),
        ('NULL', 'short'),
        ('unkn2', 'int'),
        ('color', 'ubyte[4]'), # RGBA
        ('unkn4', 'float'),
        ('area', 'float[2]'),
        ('bright', 'int'),
        ('unkn5', 'float[9]')
    ])

class Follow(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'float[23]'),
        ('unkn1', 'int[2]'),
        ('unkn2', 'float[3]')
    ])

    
class Tex_Block(PyCStruct):
    
    class Tex_Set(PyCStruct):

        class Tex_Type(PyCStruct):

            fields = OrderedDict([])

            def __init__(self, data=None, **kw):
                self.tex_type = kw.get('tex_type', 0x15)
    #             if self.tex_type not in [0x80, 0x06, 0x03,0x0A,0x0C, 0x15]:
    #             print(self.tex_type)
                if self.tex_type == 0x80:
                    self.fields = OrderedDict([
                        ('head', 'long'),
                        ('NULL', 'long'),
                        ('path_len', 'int')
                    ])
                elif self.tex_type == 0x06:
                    self.fields = OrderedDict([
                        ('NULL', 'int64'),
                        ('unkn', 'int')
                    ])
                elif self.tex_type in [0x03,0x0A,0x0C]:
                    self.fields = OrderedDict([
                        ('NULL', 'long[3]'),
                    ])
                elif self.tex_type == 0x15:
                    self.fields = OrderedDict([('unkn', 'float[6]')])

                super().__init__(data, **kw)

                if self.tex_type == 0x80:
                    self.path = data.read(self.path_len).decode('utf-8')

            def serialize(self):
                e_path = b''
                if self.tex_type == 0x80:
                    (e_path, self.path_len) = phrase_EOS(self.path, self.path_len)
                return super().serialize() + e_path

        fields = OrderedDict([
            ('set', 'long'),
            ('unkn0', 'int'),
            ('t', 'long'),
            ('textype', 'int')
        ])

        def __init__(self, data=None, **kw):
            super().__init__(data, **kw)
            self.tex_type = Tex_Block.Tex_Set.Tex_Type(data, tex_type=self.textype)

        def serialize(self):
            return super().serialize() + self.tex_type.serialize()
    
    fields = OrderedDict([
        ('Block_Header', 'long[2]'),
        ('unkn03','long'),
        ('Set_Count','int')
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.tex_set = [Tex_Block.Tex_Set(data) for i in range(self.Set_Count)]
        
    def serialize(self):
        return super().serialize() + b''.join([i.serialize() for i in self.tex_set])

class Texture(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn00', 'int64'),
        ('Block_Count', 'int'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.tex_block = [Tex_Block(data) for i in range(self.Block_Count)]
        
    def serialize(self):
        return super().serialize() + b''.join([i.serialize() for i in self.tex_block])


class CM_Map(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('path_len', 'int'),
        ('path', 'char'),
        ('unkn1', 'float[3]'),
        ('unkn_set_1', 'float[6]'),  # XYZ
        ('NULL', 'int64[3]'),
        ('unkn2', 'float[12]'),
        ('unkn_set_2', 'float[6]'),  # XYZ
        ('unkn3', 'float[5]')
    ])
    
    def __init__(self, data=None, **kw):
        self.fields['path'] = 'char[{}]'.format(struct.unpack('i'*3, _PEEK(data,12))[2])
        super().__init__(data, **kw)
        
    def serialize(self):
        if self.path and self.path[-1] != '\0':
            self.path += '\0'
        self.path_len = len(self.path)
        return super().serialize()

    
class Unkn_01(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn', 'long'),
        ('unkn2', 'float[4]')
    ])

class Unkn_03(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0','int'),
        ('length','int'),
        ('unkn1', 'long[88]'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        n = _PEEK(data,256).find(0)+1
        self.path = data.read(n).decode('utf-8')
    
    def serialize(self):
        return super().serialize() + phrase_EOS(self.path, 1)[0]

class Unkn_04(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('NULL', 'long'),
        ('unkn0', 'int'),
        ('unkn1', 'long'),
        ('float_set1', 'float[3]'),  # XYZ
        ('NULL0', 'long'),
        ('float_set2', 'float[3]'),  # XYZ
        ('NULL1', 'long')
    ])

    
class Unkn_06(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int'),
        ('NULL0', 'int64[3]'),
        ('unkn1', 'float[43]'),
        ('unkn2', 'int'),
        ('unkn3', 'float[8]')
    ])

class Unkn_07(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'float[4]'),
        ('NULL', 'int64'),
        ('unkn2', 'int[2]')
    ])

class Unkn_11(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn0', 'int[2]'),
        ('unkn1', 'float[8]')
    ])

class Unkn_12(PyCStruct):
    fields = OrderedDict([
        ('type', 'long'),
        ('unkn', 'short[10]')
    ])

        

In [33]:
class Unkn_26(PyCStruct):
    fields = OrderedDict([
        ('type' ,'long'),
        ('unkn0' ,'int[2]'),
        ('unkn1' ,'long[4]'),
    ])
    
    def __init__(self, data=None, **kw):
        super().__init__(data, **kw)
        self.type_s3b = Single_Prefix.S_3.S_3_Block(data)
        
    def serialize(self):
        return super().serialize() + self.type_s3b.serialize()

# End EFX

In [34]:
class NULL_Entry(PyCStruct):
    fields = OrderedDict([
        ('NULL_Header', 'long'),
        ('unkn0', 'long[3]'),
        ('Entry_Count', 'int'),
    ])
    def __init__(self, data, **kw):
        super().__init__(data, **kw)
        self.Entry = struct.unpack('i'*self.Entry_Count,data.read(4*self.Entry_Count))
    def serialize(self):
        self.Entry_Count = len(self.Entry)
        return super().serialize() + b''.join([struct.pack('i', entry) for entry in self.Entry])

# Test Area

In [65]:
zlib.crc32(x)

3381084852

# Note
https://blender.stackexchange.com/questions/144104/texture-baking-problem
https://blender.stackexchange.com/questions/119352/solved-trying-to-bake-texture-from-multiple-materials-blender-baking-uv-text

In [76]:
path

'chunk/vfx/efx/cm_TU/cm_emocom/emocom_000.efx'

In [114]:
file = 'acc030_003.efx'
root = 'chunk/'

path = find_file(file, root)

efx = EFX(open(path, 'rb'))
efxs = efx.serialize()
cmp = open(path, 'rb').read()

efxs == cmp

chunk/vfx/efx/cm/cm_acc/acc030/acc030_003.efx
OrderedDict([('unkn0', 'long[21]')])


True

In [350]:
eout = open('out.efx', 'wb')
eout.write(efx.serialize())
eout.close()